In [56]:
import joblib
import pandas as pd
from better_profanity import profanity

players = pd.read_csv("data/players.csv")

player_rounds = pd.read_csv("data/player_rounds.csv")

valid_ids = pd.read_csv("data/valid_ids.csv")

push_stats = pd.read_csv("data/push_stats.csv")

teams = pd.read_csv("data/teams.csv")

In [ ]:
names = players['name'].copy().loc[1:1000]

profanity.load_censor_words()

players['censor'] = names.apply(profanity.contains_profanity)

TypeError: cannot unpack non-iterable bool object

### Player Overview Tables ###

* Basic Stats Long

In [57]:
normal_cols = ['kills','deaths','assists','dmg','dmg_real','dt','dt_real','hr']
pct_cols = ['kill_pct','deaths_pct','assists_pct','dmg_pct','dmg_real_pct','dt_pct','dt_real_pct','hr_pct']

both_cols = ['id','team','steamid','name','primary_class']

normal_df = players[both_cols + normal_cols].copy()
pct_df = players[both_cols + pct_cols].copy()

pct_df.columns = normal_df.columns

normal_df['coltype'] = "Raw"
pct_df['coltype'] = "Pct of Team"


long_player = pd.concat([normal_df,pct_df])

long_player = long_player.fillna(0)

long_player.to_csv("data/long_player_stats.csv")

* Class KDA Long

In [58]:
class_cols = [col for col in players.columns if "class_kda" in col]

class_kill = [col for col in class_cols if "kills" in col and "kdapd" not in col]

class_deaths= [col for col in class_cols if "deaths" in col and "kdapd" not in col]

class_assists = [col for col in class_cols if "assists" in col and "kdapd" not in col]

class_kill_pd = [col for col in class_cols if "kills" in col and "kdapd" in col]

class_deaths_pd = [col for col in class_cols if "deaths" in col and "kdapd" in col]

class_assists_pd = [col for col in class_cols if "assists" in col and "kdapd" in col]

keep_cols = ['id','team','steamid','name','primary_class']



class_dict = {
    "Kills" : class_kill,
    "Deaths" : class_deaths,
    "Assists" : class_assists,
    "K/D" : class_kill_pd,
    "A/D" : class_assists_pd,
    "Death Rate" : class_deaths_pd
}

long_class = pd.DataFrame()
for key in class_dict.keys():
    col_type = key
    l = class_dict[key]
    df = players[keep_cols + l].copy()
    df.columns = keep_cols + [col.split("_")[0] for col in df.columns if col not in keep_cols]
    df['col_type'] = col_type
    long_class = pd.concat([long_class,df])

long_class = long_class.fillna(0)

long_class.to_csv("data/class_kda_long.csv")

* Attach player names to player_rounds

In [64]:
names = players[['id','steamid','name']]

player_rounds = player_rounds.merge(names, on = ['id','steamid']).drop("Unnamed: 0",axis= 1)

player_rounds.to_csv("data/player_rounds.csv")

In [52]:
long_class.fillna(0)

,id,team,steamid,name,primary_class,demoman,heavyweapons,medic,pyro,scout,sniper,soldier,engineer,spy,col_type
0,3886768,Red,[U:1:113283780],loafe,medic,0.0000,0.0,0.0,0.0000,0.00,0.0000,0.0000,0.0000,0.0000,Kills
1,3886768,Blue,[U:1:131704728],sushi,medic,0.0000,0.0,0.0,0.0000,0.00,0.0000,0.0000,0.0000,0.0000,Kills
2,3886768,Blue,[U:1:136473263],Math,soldier,4.0000,0.0,0.0,1.0000,3.00,0.0000,1.0000,0.0000,0.0000,Kills
3,3886768,Blue,[U:1:159413592],TIGER BALM,scout,1.0000,0.0,0.0,0.0000,2.00,0.0000,8.0000,0.0000,0.0000,Kills
4,3886768,Red,[U:1:185989865],gabby,soldier,3.0000,0.0,1.0,0.0000,2.00,1.0000,6.0000,0.0000,0.0000,Kills
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53492,1764254,Red,[U:1:44119927],Kevin | Gas Station Soda - 99¢,pyro,0.0000,0.0,0.0,0.3333,0.00,0.3333,0.0000,0.3333,0.0000,Death Rate
53493,1764254,Red,[U:1:58753664],Ambluj,spy,0.0000,0.0,0.0,0.3333,0.00,0.0000,0.3333,0.1667,0.1667,Death Rate
53494,1764254,Blue,[U:1:78449395],Kevin | ProfessorParsnips,sniper,0.0000,0.0,0.0,0.0000,0.00,0.0000,0.0000,0.0000,1.0000,Death Rate
53495,1764254,Blue,[U:1:82268202],Serru,scout,0.0000,0.0,0.0,0.2500,0.75,0.0000,0.0000,0.0000,0.0000,Death Rate


['demoman',
 'engineer',
 'heavyweapons',
 'pyro',
 'scout',
 'sniper',
 'soldier',
 'medic',
 'spy']

In [30]:
pd.Series(class_kill).sort_values()

0         demoman_kills_class_kda
7        engineer_kills_class_kda
1    heavyweapons_kills_class_kda
2           medic_kills_class_kda
3            pyro_kills_class_kda
4           scout_kills_class_kda
5          sniper_kills_class_kda
6         soldier_kills_class_kda
8             spy_kills_class_kda
dtype: object

In [2]:
[col for col in players.columns if "pd" in col]

['dapd',
 'kapd',
 'kpd',
 'assistspd',
 'demoman_kills_class_kdapd',
 'heavyweapons_kills_class_kdapd',
 'medic_kills_class_kdapd',
 'pyro_kills_class_kdapd',
 'scout_kills_class_kdapd',
 'sniper_kills_class_kdapd',
 'soldier_kills_class_kdapd',
 'demoman_deaths_class_kdapd',
 'engineer_deaths_class_kdapd',
 'heavyweapons_deaths_class_kdapd',
 'pyro_deaths_class_kdapd',
 'scout_deaths_class_kdapd',
 'sniper_deaths_class_kdapd',
 'soldier_deaths_class_kdapd',
 'demoman_assists_class_kdapd',
 'heavyweapons_assists_class_kdapd',
 'medic_assists_class_kdapd',
 'pyro_assists_class_kdapd',
 'scout_assists_class_kdapd',
 'sniper_assists_class_kdapd',
 'soldier_assists_class_kdapd',
 'engineer_kills_class_kdapd',
 'spy_kills_class_kdapd',
 'medic_deaths_class_kdapd',
 'spy_deaths_class_kdapd',
 'engineer_assists_class_kdapd',
 'spy_assists_class_kdapd']

In [ ]:
# List out columns
quant_cols = [
    "kills",
    "assists",
    "dmg",
   'dapm',
   'kpd',
   'offclass_pct',
   'kill_pct',
   'deaths_pct',
   'dmg_pct',
   'dmg_real_pct',
   'cpc_pct',
   'ka_pct',
   'assists_pct',
   'dt_pct',
   'dt_real_pct',
   'hroi',
   'assistspd',
   'demoman_kills_class_kdapd',
   'scout_kills_class_kdapd',
   'soldier_kills_class_kdapd',
   'medic_kills_class_kdapd',
    'demoman_deaths_class_kdapd',
   'scout_deaths_class_kdapd',
   'soldier_deaths_class_kdapd',
    'medic_deaths_class_kdapd',
   'dtpm',
   'dt_realpm',
   'healpm',
   'medkits_hppm', 
   'hrpm', 
   'deathspm'

]

# Only grab valid ids to make quanitles on
sub_players = players[players['id'].isin(valid_ids['id'])].copy()
sub_players = sub_players[['id',"primary_class",'steamid']+ quant_cols]

# loop through classes and construct the quantile sets
ranked_df = pd.DataFrame()
for class_name in sub_players['primary_class'].unique():
    sub_class = sub_players[sub_players['primary_class'] == class_name].copy()
    
    binding_df = sub_class[['id',"primary_class",'steamid']].copy()
    sub_class.drop(['id',"primary_class",'steamid'],axis = 1,inplace= True)
    
    sub_class = sub_class.rank(pct = True)
    sub_class = pd.concat([binding_df,sub_class],axis = 1)
    
    ranked_df = pd.concat([ranked_df,sub_class])
    
# Rename columns
ranked_df.columns = ['id','primary_class','steamid'] + [col + '_quantile' for 
                                                        col in ranked_df.columns if
                                                        col not in ['id','steamid','primary_class']]

ranked_df.to_csv("data/players_quantile.csv")
